This file will syncronise with eye tracking data to create event markers first. Then it will clean and process eeg raw data in proper data format. It will then be clustered and labelled into 3 states :{ Attention, Neutral, Rejection } . Then, It will re-syncronise with the eye tracking data to create a new eye tracking with labels which would contain both eye tracking and eeg labels. Afterwhich, It runs a processing method to compute saliency for attention and preference based on the eeg and eye tracking data. The points with the saliency score is the hitpoint data. Additional steps will be required to impose hitpoint saliency onto mesh points. The output file will ultimately be Point Cloud with saliency scores.



In [ ]:
#install dependencies
!pip install autoreject
!pip install mne
!pip install pyedflib
!pip install --upgrade mne

The code below concatenates easy files if required

In [ ]:
#Concatenate eeg files if needed, otherwise skip this step 
import pandas as pd

# Paths to your .easy files
file1 = ""
file2 = ""

# Load both files into pandas DataFrames
try:
    df1 = pd.read_csv(file1, sep='\t', header=None)  # Adjust header if the files have headers
    df2 = pd.read_csv(file2, sep='\t', header=None)

    combined_df = pd.concat([df1, df2], ignore_index=True)

   
    print("Files combined successfully. Here's a preview:")
    print(combined_df.head())  
    print("Last few rows of the combined DataFrame:")
    print(combined_df.tail()) 
except Exception as e:
    print(f"Error combining files: {e}")


The code below syncs eye tracking and eeg files to mark events on the eeg file

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta


file_path = "" ## eeg file 
data = pd.read_csv(file_path, delimiter="\t", dtype={'column_name': 'category', 'another_column': 'float64'})

num_columns = data.shape[1]
headers = [f'channel {i+1}' for i in range(num_columns - 5)] + ['aX', 'aY', 'aZ','markers', 'timestamp']

data.columns = headers
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')

data['timestamp'] = pd.to_datetime(data['timestamp'], format='%d/%m/%Y %I:%M:%S %p')
#scaling factor as eeg raw data is in nanovolts but processing of eeg is in microvolts
scaling_factor = 1e-3

eeg_channels = [col for col in data.columns if 'channel' in col]
for col in eeg_channels:
    data[col] = data[col] * scaling_factor
output_csv_path = "output_data.csv"
data.to_csv(output_csv_path, index=False)
print(f"Data saved to {output_csv_path}")
eeg_data = data
eeg_data['timestamp'] = pd.to_datetime(eeg_data['timestamp'])
eeg_data['markers'] = 0

for n in range(1, 16):  
    for shape in ['curved', 'rect']:
        eye_file = f"/eyetrackingdata__{shape}{n}.csv" ##csv for eye tracking to sync timestamp 
        try:
            eye_data = pd.read_csv(eye_file)
            eye_data['Timestamp'] = pd.to_datetime(eye_data['Timestamp'], errors='coerce')
            eye_data = eye_data.drop_duplicates().sort_values(by='Timestamp').reset_index(drop=True)
            eye_data['Timestamp'] -= timedelta(hours=8)  # Adjust timezone

            eeg_timestamps = eeg_data['timestamp'].to_numpy()
            eye_timestamps = eye_data['Timestamp'].to_numpy()

            closest_indices = np.searchsorted(eeg_timestamps, eye_timestamps)
            closest_indices = np.clip(closest_indices, 1, len(eeg_timestamps) - 1)
            left_diffs = np.abs(eeg_timestamps[closest_indices - 1] - eye_timestamps)
            right_diffs = np.abs(eeg_timestamps[closest_indices] - eye_timestamps)

            best_indices = np.where(left_diffs <= right_diffs, closest_indices - 1, closest_indices)

            eeg_data.loc[best_indices, 'markers'] = 1

            print(f"Processed and synced file: {eye_file}")

        except Exception as e:
            print(f"Error processing file {eye_file}: {e}")

# please do not use this file for processing as excel has limited rows so some data is lost in this file. Use the stored dataframe instance. This file is only used for visual purpose. 
output_file = "compiled_synced_eeg_data.csv"
eeg_data.to_csv(output_file, index=False)
print(f"Final synchronized EEG data saved to: {output_file}")

The code below checks if it is properly synced

In [ ]:
total_rows = 0

for n in range(1, 16):  
    for shape in ['curved', 'rect']:
        eye_file = f"/eyetrackingdata__{shape}{n}.csv"

        try:
            eye_data = pd.read_csv(eye_file)
            total_rows += len(eye_data)

        except Exception as e:
            print(f"Error processing file {eye_file}: {e}")

print(f"Total number of rows across all eye-tracking files: {total_rows}")

In [ ]:
# Count the number of rows where markers == 1
num_markers = (eeg_data['markers'] == 1).sum()

print(f"Number of markers set to 1: {num_markers}")
print(eeg_data.head())

The code below looks at the visualisation of eeg signals after processing methods like filtering and ica removal 

In [ ]:
## visualisation of eeg signals after processing. Check the efficacy of filtering and cleaning methods
import mne
import numpy as np
import matplotlib.pyplot as plt

sfreq = 256  # Sampling frequency
eeg_channels = ["channel 1", "channel 2", "channel 7", "channel 8"]  # EEG channels
ch_names = eeg_channels + ["markers"]
ch_types = ["eeg"] * len(eeg_channels) + ["stim"]

# Extract and transpose data
raw_data = data[eeg_channels + ["markers"]].to_numpy().T

# Create MNE RawArray
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(raw_data, info)

mapping = {
    "channel 1": "F7",
    "channel 2": "AF7",
    "channel 7": "AF8",
    "channel 8": "F8"
}
raw.rename_channels(mapping)

montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage)

raw.filter(0.2, 35, fir_design="firwin") 
raw.notch_filter(freqs=50, method="spectrum_fit")  

# Perform ICA for EOG artifact removal
ica = mne.preprocessing.ICA(n_components=4, random_state=97, max_iter=800)
ica.fit(raw)

# Plot ICA components to visually inspect which component corresponds to EOG artifact
ica.plot_components()

# Assuming 'eog_inds' are indices of ICA components corresponding to EOG artifacts
eog_inds, scores = ica.find_bads_eog(raw, ch_name="AF8")  # Use relevant channel to find EOG components

print("Identified EOG components:", eog_inds)

# Remove EOG artifacts by excluding the identified components
ica.exclude = eog_inds
raw_clean = ica.apply(raw)  # Apply ICA to remove artifacts

# Plot the EEG data before and after ICA cleaning (using ICA overlay)
ica.plot_overlay(raw, exclude=[0], picks="eeg", title="Before and After ICA Cleaning")


# Show the plot with the legend
plt.show()

The code below incoporates filtering, clustering and labelling

In [ ]:
## Using Kmeans and Event Related Potential processing to cluster and label eeg signal points 
import numpy as np
import pandas as pd
import mne
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Function to extract band power
def extract_band_power(psd_data, freqs, band):
    band_indices = np.logical_and(freqs >= band[0], freqs <= band[1])
    return psd_data[:, :, band_indices].mean(axis=2)

data = eeg_data
data["timestamp"] = pd.to_datetime(data["timestamp"], errors="coerce")
data.dropna(subset=["timestamp"], inplace=True)
data.reset_index(drop=True, inplace=True)

# Metadata
sfreq = 256
eeg_channels = ["channel 1", "channel 2", "channel 7", "channel 8"]
ch_names = eeg_channels + ["markers"]
ch_types = ["eeg"] * len(eeg_channels) + ["stim"]

raw_data = data[eeg_channels + ["markers"]].to_numpy().T

# Create MNE RawArray
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(raw_data, info)

# Map channel names to standard 10-20 system
mapping = {
    "channel 1": "F7",
    "channel 2": "AF7",
    "channel 7": "AF8",
    "channel 8": "F8"
}
raw.rename_channels(mapping)

montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage)
## If possible use xdawn filtering for ERP 
raw.filter(0.2, 35, fir_design="firwin")  
raw.notch_filter(freqs=50, method="spectrum_fit") 


ica = mne.preprocessing.ICA(n_components=4, random_state=97, max_iter=800)
ica.fit(raw)

ica.plot_components()

eog_indices, scores = ica.find_bads_eog(raw, ch_name="AF7")

# Exclude the identified EOG components
ica.exclude = eog_indices

# Apply ICA to remove the artifacts
## If possible, use auto reject or channels with EOG for better removal of artifacts and noises
raw_clean = ica.apply(raw)

# Create events from cleaned raw data
valid_markers = data["markers"] == 1  
if valid_markers.any():
    events = np.array([[i, 0, 1] for i in range(len(data)) if valid_markers.iloc[i]])
    event_id = {"Event_1": 1}

    # Create epochs
    epochs = mne.Epochs(
        raw_clean, events, event_id=event_id, tmin=-0.2, tmax=0.9, preload=True
    )

    # Compute PSD for the epochs
    spectrum = epochs.compute_psd(
        method="welch", fmin=0.5, fmax=40, n_fft=128
    )
    psd_data, freqs = spectrum.get_data(return_freqs=True)

    # Feature extraction: P300 (Attention), N400 (Rejection), and LPP (Neutral)
    p300_band = (10, 20)  # Attention
    n400_band = (3, 6)   # Rejection
    lpp_band = (1, 3)    # Neutral
    
    p300_amplitude = extract_band_power(psd_data, freqs, p300_band)
    n400_amplitude = extract_band_power(psd_data, freqs, n400_band)
    lpp_amplitude = extract_band_power(psd_data, freqs, lpp_band)

    # Stack features for clustering
    features = np.vstack((p300_amplitude.mean(axis=1), n400_amplitude.mean(axis=1), lpp_amplitude.mean(axis=1))).T

    # KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    cluster_labels = kmeans.fit_predict(features)

    # Compute silhouette score
    silhouette_avg = silhouette_score(features, cluster_labels)
    print(f"Silhouette Score: {silhouette_avg}")

    # Map clusters to emotional states
    cluster_map = {
        0: 'Attention',     # High P300
        1: 'Rejection',     # High N400
        2: 'Neutral'        # High LPP
    }

    epoch_labels = [cluster_map[label] for label in cluster_labels]

    data["label"] = "Neutral"  
    for i, label in enumerate(epoch_labels):
        epoch_start = events[i, 0] 
        data.at[epoch_start, "label"] = label

    output_path = "labeled_eeg_data.csv"
    data.to_csv(output_path, index=False)
    print(f"Labeled data saved to {output_path}")

else:
    print("No valid markers found.")

label_counts = data["label"].value_counts()
print("Summary of classifications:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

The code below maps back eeg with labels with eye tracking data

In [ ]:
##Re sync the eeg events with eye tracking on a new file eye tracking with labels
import pandas as pd
import numpy as np
from datetime import timedelta

eeg_data = data
eeg_data['timestamp'] = pd.to_datetime(eeg_data['timestamp'])

eeg_data['label'] = eeg_data['label'].fillna('Neutral')  # Fill missing labels with 'Neutral'

for n in range(1, 16):  ## range of ur file 
    for shape in ['curved', 'rect']:
        eye_file = f"/eyetrackingdata__{shape}{n}.csv"

        try:
            eye_data = pd.read_csv(eye_file)

            eye_data['Timestamp'] = pd.to_datetime(eye_data['Timestamp'], errors='coerce')
            eye_data = eye_data.drop_duplicates().sort_values(by='Timestamp').reset_index(drop=True)

            # Add 8 hours to adjust for local time (if needed)
            eye_data['Timestamp'] = eye_data['Timestamp'] - timedelta(hours=8)

            eeg_timestamps = eeg_data['timestamp'].to_numpy()
            eye_timestamps = eye_data['Timestamp'].to_numpy()

            closest_indices = np.searchsorted(eeg_timestamps, eye_timestamps)

            closest_indices = np.clip(closest_indices, 1, len(eeg_timestamps) - 1)
            left_diffs = np.abs(eeg_timestamps[closest_indices - 1] - eye_timestamps)
            right_diffs = np.abs(eeg_timestamps[closest_indices] - eye_timestamps)

            best_indices = np.where(left_diffs <= right_diffs, closest_indices - 1, closest_indices)

            eye_data['label'] = eeg_data.loc[best_indices, 'label'].values

            output_file = f"eye_tracking_with_labels_{shape}{n}.csv"
            eye_data.to_csv(output_file, index=False)
            print(f"Processed and labeled file: {eye_file}")

        except Exception as e:
            print(f"Error processing file {eye_file}: {e}")

print("Labeling process complete.")

The code below checks for if eeg is merged well with eye tracking

In [ ]:
import pandas as pd

total_rejection = 0
total_attention = 0
total_neutral = 0

file_summary = []

## Do a quick visualisation if files are sync correctly. The numbers should tally. 
for n in range(1, 16):  
    for shape in ['curved', 'rect']:
        output_file = f"/eye_tracking_with_labels_{shape}{n}.csv"
        try:
            labeled_eye_data = pd.read_csv(output_file)
            label_counts = labeled_eye_data['label'].value_counts()

            rejection_count = label_counts.get('Rejection', 0)
            attention_count = label_counts.get('Attention', 0)
            neutral_count = label_counts.get('Neutral', 0)

            total_rejection += rejection_count
            total_attention += attention_count
            total_neutral += neutral_count

            file_summary.append({
                'File': output_file,
                'Rejection': rejection_count,
                'Attention': attention_count,
                'Neutral': neutral_count
            })

        except Exception as e:
            print(f"Error reading file {output_file}: {e}")

summary_df = pd.DataFrame(file_summary)

print("Per-File Label Counts:")
print(summary_df)

print("\nTotal Counts Across All Files:")
print(f"Total Rejection: {total_rejection}")
print(f"Total Attention: {total_attention}")
print(f"Total Neutral: {total_neutral}")

The code below incoporates eeg and eye tracking data with eye tracking labelled data to compute attention and preference saliency. It is then combined through Sum and Multiply combined scores and outputed 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import CheckButtons

############################################################################################################################################
df1 = pd.read_csv("/eye_tracking_with_labels_{shape}{n}.csv") ## files with eye tracking and eeg labels

## mesh data
df2 = pd.read_csv("/{shape}{n}_points.csv") ## files with point cloud 
############################################################################################################################################
x_min, x_max = df2['x'].min(), df2['x'].max()
y_min, y_max = df2['y'].min(), df2['y'].max()
z_min, z_max = df2['z'].min(), df2['z'].max()

print("Mesh Points Boundaries:")
print(f"X range: [{x_min:.4f}, {x_max:.4f}]")
print(f"Y range: [{y_min:.4f}, {y_max:.4f}]")
print(f"Z range: [{z_min:.4f}, {z_max:.4f}]")

filtered_df1 = df1[
    (df1['HitPointX'] >= x_min) & (df1['HitPointX'] <= x_max) &
    (df1['HitPointY'] >= y_min) & (df1['HitPointY'] <= y_max) &
    (df1['HitPointZ'] >= z_min) & (df1['HitPointZ'] <= z_max) &
    (df1['HitObject'] != 'None')

]
    

print(f"\nOriginal raw data points: {len(df1)}")
print(f"Points after filtering: {len(filtered_df1)}")
print(f"Mesh points: {len(df2)}")

plt.rcParams['figure.figsize'] = [15, 10]
fig = plt.figure()
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.95)

ax = fig.add_subplot(111, projection='3d')

raw_data_plot = ax.scatter(filtered_df1['HitPointX'],
                          filtered_df1['HitPointY'],
                          filtered_df1['HitPointZ'],
                          c='blue', marker='o', label='Raw Data (filtered)',
                          alpha=0.6, s=1)

mesh_points_plot = ax.scatter(df2['x'], df2['y'], df2['z'],
                            c='red', marker='^', label='Mesh points',
                            alpha=0.6, s=5)


ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('All Points Visualization\nFiltered Raw Data and Mesh Points')
ax.grid(True)
ax.view_init(elev=20, azim=45)

ax_check = plt.axes([0.02, 0.05, 0.15, 0.10])
check = CheckButtons(ax_check, ['Raw Data', 'Mesh Points'],
                    [True, True])  # Both visible initially

import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

eye_data = filtered_df1.copy()
eye_data.reset_index(drop=True, inplace=True)

# Convert Timestamp to datetime format
eye_data['Timestamp'] = pd.to_datetime(eye_data['Timestamp'])
eye_data = eye_data.sort_values(by='Timestamp')
eye_data = eye_data.drop_duplicates(keep='first')
# Check for any conversion issues
eye_data.reset_index(drop=True, inplace=True)
if eye_data['Timestamp'].isnull().any():
    print("Some timestamps could not be converted. Check the data for invalid formats.")


mesh_data = df2.copy()

def cluster_eye_data(eye_data, normal_tolerance=0.001):
    for i in range(len(eye_data) - 1):
        current_row = eye_data[i]
        next_row = eye_data[i + 1]

        # Check gaze normal difference between current row and next row
        normal_diff_next = check_normal_difference(current_row['NormalX'], current_row['NormalY'], current_row['NormalZ'],
                                                   next_row['NormalX'], next_row['NormalY'], next_row['NormalZ'],
                                                   normal_tolerance)

        # If gaze normals are similar, update the current row's hit point
        if normal_diff_next:
            current_row['HitPointX'] = next_row['HitPointX']
            current_row['HitPointY'] = next_row['HitPointY']
            current_row['HitPointZ'] = next_row['HitPointZ']

    return eye_data


def check_normal_difference(normal1_x, normal1_y, normal1_z, normal2_x, normal2_y, normal2_z, tolerance):
    ##cluster neighbouring points for more concentrated data
    diff_x = abs(normal1_x - normal2_x)
    diff_y = abs(normal1_y - normal2_y)
    diff_z = abs(normal1_z - normal2_z)

    # If the difference in all components is less than or equal to the tolerance, return True
    return diff_x <= tolerance and diff_y <= tolerance and diff_z <= tolerance

def check_normal_difference(normal1_x, normal1_y, normal1_z, normal2_x, normal2_y, normal2_z, tolerance):
    # Calculate the difference between the two gaze normals in each component
    diff_x = abs(normal1_x - normal2_x)
    diff_y = abs(normal1_y - normal2_y)
    diff_z = abs(normal1_z - normal2_z)

    # If the difference in all components is less than or equal to the tolerance, return True
    return diff_x <= tolerance and diff_y <= tolerance and diff_z <= tolerance

def calculate_eye_tracking_metrics(data, fixation_threshold=0.1):
    metrics_list = []
    visited_points = {}
    data['InitialSaliency'] = 0

    for i, row in data.iterrows():
        current_point = (row['HitPointX'], row['HitPointY'], row['HitPointZ'])
        timestamp = row['Timestamp']
        label = row['label']

        if current_point not in visited_points:
            visited_points[current_point] = {
                'FixationCount': 0,
                'DwellTime': 0,  # Accumulate total dwell time for the point
                'RevisitCount': 0,
                'TTFF': None,  # Initialize Time to First Fixation as None
                'CumulativeDwell': 0,  # Track cumulative dwell time for fixation threshold
                'FirstFixationTime': None,  # To track when first fixation happens
                'Labels': set()  # Store unique labels for this point
            }

        visited_points[current_point]['Labels'].add(label)

        if i < len(data) - 1:
            next_row = data.iloc[i + 1]
            next_timestamp = next_row['Timestamp']
            dwell_time = (next_timestamp - timestamp).total_seconds()
            visited_points[current_point]['DwellTime'] += dwell_time

            # Accumulate CumulativeDwell only if the current and next points are the same
            next_point = (next_row['HitPointX'], next_row['HitPointY'], next_row['HitPointZ'])
            if current_point == next_point:
                visited_points[current_point]['CumulativeDwell'] += dwell_time
            else:
                # Reset CumulativeDwell if the current point does not match the next point
                visited_points[current_point]['CumulativeDwell'] = 0

        # Time to First Fixation (TTFF) logic: set only the first time the fixation threshold is exceeded
        if visited_points[current_point]['FirstFixationTime'] is None and visited_points[current_point]['CumulativeDwell'] >= fixation_threshold:
            visited_points[current_point]['FirstFixationTime'] = timestamp
            visited_points[current_point]['TTFF'] = (timestamp - data['Timestamp'].min()).total_seconds()

        # Fixation Count: Increment each time the cumulative dwell time exceeds the threshold
        if visited_points[current_point]['CumulativeDwell'] >= fixation_threshold:
            visited_points[current_point]['FixationCount'] += 1
            visited_points[current_point]['CumulativeDwell'] = 0  

        # Revisit Count: Increment when the same point appears consecutively in the data
        if i > 0:
            previous_point = (data.loc[i - 1, 'HitPointX'], data.loc[i - 1, 'HitPointY'], data.loc[i - 1, 'HitPointZ'])
            if current_point == previous_point:
                visited_points[current_point]['RevisitCount'] += 1

    for point, metric in visited_points.items():
        metrics_list.append({
            'HitPointX': point[0],
            'HitPointY': point[1],
            'HitPointZ': point[2],
            'FixationCount': metric['FixationCount'],
            'DwellTime': metric['DwellTime'],
            'RevisitCount': metric['RevisitCount'],
            'TTFF': metric['TTFF'] if metric['TTFF'] is not None else 0,
            'Labels': tuple(metric['Labels'])  # Convert the set to a tuple
        })

    metrics_df = pd.DataFrame(metrics_list)

    # Normalize metrics with inverse normalization for TTFF
    for col in ['FixationCount', 'DwellTime', 'RevisitCount']:
        if metrics_df[col].max() != 0:  # Avoid division by zero
            metrics_df[col] = metrics_df[col] / metrics_df[col].max()

    # Inverse normalization for TTFF: lower TTFF is more salient
    if metrics_df['TTFF'].min() != 0:
        metrics_df['TTFF'] = 1 - (metrics_df['TTFF'] / metrics_df['TTFF'].max())
    else:
        metrics_df['TTFF'] = 0

    metrics_df['InitialSaliency'] = (
        0.1 +
        0.2 * metrics_df['FixationCount'] +
        0.3 * metrics_df['DwellTime'] +
        0.2 * metrics_df['RevisitCount'] +
        0.2 * metrics_df['TTFF']
    )

    metrics_df['RevisitCount'] = metrics_df['RevisitCount'].astype(int)

    return metrics_df


# Example usage assuming eye_data DataFrame has a 'EEGLabel' column
eye_metrics = calculate_eye_tracking_metrics(eye_data)

def normalize_saliency_score(data):
    """
    Normalize the InitialSaliency score by dividing it by its maximum value.
    The result will be a saliency score between 0 and 1.
    """
    max_saliency = data['InitialSaliency'].max()

    if max_saliency != 0:
        data['InitialSaliency'] = data['InitialSaliency'] / max_saliency
    else:
        data['InitialSaliency'] = 0

    return data
eye_metrics = calculate_eye_tracking_metrics(eye_data)

# Normalize the InitialSaliency score
eye_metrics = normalize_saliency_score(eye_metrics)

import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

mesh_points = df2.copy()
N = len(mesh_points)

alpha = 0.5

average_spacing = 1 / np.cbrt(N)

# Calculate mesh volume based on x, y, and z ranges
mesh_volume = (mesh_points['x'].max() - mesh_points['x'].min()) * \
              (mesh_points['y'].max() - mesh_points['y'].min()) * \
              (mesh_points['z'].max() - mesh_points['z'].min())

# Calculate initial and adjusted radius based on mesh volume normalization
radius = average_spacing * (alpha ** (1 / 3))
adjusted_radius = radius * np.sqrt(mesh_volume / (N * (4 / 3) * np.pi * radius**3))

print(f"Adjusted Radius: {adjusted_radius}")

k = 0.4  # Weight for saliency scores

# KDTree preparation for eye points and mesh points
eye_points = eye_metrics[['HitPointX', 'HitPointY', 'HitPointZ']].values
mesh_points_coords = mesh_points[['x', 'y', 'z']].values
eye_tree = cKDTree(eye_points)

# Initialize columns
mesh_points['SaliencyScore'] = 0.0
mesh_points['EEGScore'] = 0.0
mesh_points['Labels'] = [[] for _ in range(len(mesh_points))]

# Define label scores 
label_scores = {'Attention': 1, 'Rejection': -1, 'Neutral': 0}

# Query the mesh points against the eye hit points (remove distance_upper_bound)
distances, indices = eye_tree.query(mesh_points_coords, k=len(eye_points))  # Query all eye points for each mesh point

# Process each mesh point to capture the valid neighbors within the adjusted radius
for i, (dist, idx) in enumerate(zip(distances, indices)):
    # Filter out the indices where the distance exceeds the adjusted radius
    valid_neighbors = idx[dist <= adjusted_radius]

    

    if len(valid_neighbors) > 0:
        try:
            # Sum the InitialSaliency for valid neighbors
            saliency_sum = eye_metrics.iloc[valid_neighbors]['InitialSaliency'].sum()
            mesh_points.at[i, 'SaliencyScore'] = saliency_sum

            # Collect labels from valid neighbors
            neighbor_labels = eye_metrics.iloc[valid_neighbors]['Labels'].tolist()

            # Flatten and collect all labels (since 'Labels' is a set or list for each point)
            flat_labels = [label for label_set in neighbor_labels for label in label_set]

            # Sum the EEG scores based on label mapping
            eeg_sum = sum(label_scores.get(label, 0) for label in flat_labels)
            mesh_points.at[i, 'EEGScore'] = eeg_sum

            # Append the labels to the 'Labels' column for the current mesh point
            mesh_points.at[i, 'Labels'] = flat_labels
        except IndexError:
            mesh_points.at[i, 'SaliencyScore'] = 0.0
            mesh_points.at[i, 'EEGScore'] = 0.0
            mesh_points.at[i, 'Labels'] = []
    else:
        mesh_points.at[i, 'SaliencyScore'] = 0.0
        mesh_points.at[i, 'EEGScore'] = 0.0
        mesh_points.at[i, 'Labels'] = []
#calculate normalised eye saliency 
min_score, max_score = mesh_points['SaliencyScore'].agg(['min', 'max'])
mesh_points['NormalizedScore'] = (mesh_points['SaliencyScore'] - min_score) / (max_score - min_score)
# Separate negative and positive values for EEGScore
negative_values_eeg = mesh_points['EEGScore'][mesh_points['EEGScore'] < 0]
positive_values_eeg = mesh_points['EEGScore'][mesh_points['EEGScore'] > 0]

max_negative_eeg = negative_values_eeg.min()  # Most negative value
max_positive_eeg = positive_values_eeg.max()  # Most positive value

# Normalize EEGScore for values less than 0 by max_negative_eeg (extreme negative)
mesh_points['NormalizedEEG'] = mesh_points['EEGScore'].apply(
    lambda x: x / abs(max_negative_eeg) if x < 0 else x / max_positive_eeg
)

# Calculate CombinedScore as the product of SaliencyScore and NormalizedEEG
mesh_points['CombinedScore'] = mesh_points['SaliencyScore'] * mesh_points['NormalizedEEG']

# Separate negative and positive values for CombinedScore
negative_values_combined = mesh_points['CombinedScore'][mesh_points['CombinedScore'] < 0]
positive_values_combined = mesh_points['CombinedScore'][mesh_points['CombinedScore'] > 0]

# Get the extreme negative and positive values for CombinedScore
max_combined_negative = negative_values_combined.min()  # Most negative combined score
max_combined_positive = positive_values_combined.max()  # Most positive combined score

# Normalize CombinedScore similarly
mesh_points['NormalizedCombinedScore'] = mesh_points['CombinedScore'].apply(
    lambda x: x / abs(max_combined_negative) if x < 0 else x / max_combined_positive
)


##########################################################################################################################################################################################################################################################################################
mesh_points[['x', 'y', 'z', 'SaliencyScore','NormalizedScore', 'Labels','EEGScore','NormalizedEEG', 'NormalizedCombinedScore']].to_csv('rect15_final_sum_combined_score.csv', index=False)
print("CSV file 'curved4_score_with_eeg.csv' has been saved.")
########################################################################################################################################################################################################################################################################################
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

mesh_points = df2.copy()
N = len(mesh_points)

alpha = 0.5

average_spacing = 1 / np.cbrt(N)

mesh_volume = (mesh_points['x'].max() - mesh_points['x'].min()) * \
              (mesh_points['y'].max() - mesh_points['y'].min()) * \
              (mesh_points['z'].max() - mesh_points['z'].min())

radius = average_spacing * (alpha ** (1 / 3))
adjusted_radius = radius * np.sqrt(mesh_volume / (N * (4 / 3) * np.pi * radius**3))

print(f"Adjusted Radius: {adjusted_radius}")

k = 0.4  # Weight for saliency scores

# KDTree preparation for eye points and mesh points
eye_points = eye_metrics[['HitPointX', 'HitPointY', 'HitPointZ']].values
mesh_points_coords = mesh_points[['x', 'y', 'z']].values
eye_tree = cKDTree(eye_points)

# Initialize columns
mesh_points['SaliencyScore'] = 0.0
mesh_points['EEGScore'] = 0.0
mesh_points['Labels'] = [[] for _ in range(len(mesh_points))]

label_scores = {'Attention': 1, 'Rejection': -1, 'Neutral': 0}

# Query the mesh points against the eye hit points (remove distance_upper_bound)
distances, indices = eye_tree.query(mesh_points_coords, k=len(eye_points))  # Query all eye points for each mesh point

# Process each mesh point to capture the valid neighbors within the adjusted radius
for i, (dist, idx) in enumerate(zip(distances, indices)):
    valid_neighbors = idx[dist <= adjusted_radius]

    if len(valid_neighbors) > 0:
        try:
            # Sum the InitialSaliency for valid neighbors
            saliency_sum = eye_metrics.iloc[valid_neighbors]['InitialSaliency'].sum()
            mesh_points.at[i, 'SaliencyScore'] = saliency_sum

            # Collect labels from valid neighbors
            neighbor_labels = eye_metrics.iloc[valid_neighbors]['Labels'].tolist()

            # Flatten and collect all labels (since 'Labels' is a set or list for each point)
            flat_labels = [label for label_set in neighbor_labels for label in label_set]

            # Sum the EEG scores based on label mapping
            eeg_sum = sum(label_scores.get(label, 0) for label in flat_labels)
            mesh_points.at[i, 'EEGScore'] = eeg_sum

            # Append the labels to the 'Labels' column for the current mesh point
            mesh_points.at[i, 'Labels'] = flat_labels
        except IndexError:
            mesh_points.at[i, 'SaliencyScore'] = 0.0
            mesh_points.at[i, 'EEGScore'] = 0.0
            mesh_points.at[i, 'Labels'] = []
    else:
        mesh_points.at[i, 'SaliencyScore'] = 0.0
        mesh_points.at[i, 'EEGScore'] = 0.0
        mesh_points.at[i, 'Labels'] = []

#calculate normalised eye saliency 
min_score, max_score = mesh_points['SaliencyScore'].agg(['min', 'max'])
mesh_points['NormalizedScore'] = (mesh_points['SaliencyScore'] - min_score) / (max_score - min_score)
# Separate negative and positive values for EEGScore
negative_values_eeg = mesh_points['EEGScore'][mesh_points['EEGScore'] < 0]
positive_values_eeg = mesh_points['EEGScore'][mesh_points['EEGScore'] > 0]

# Get the extreme negative and positive values for EEGScore
max_negative_eeg = negative_values_eeg.min()  # Most negative value
max_positive_eeg = positive_values_eeg.max()  # Most positive value

# Normalize EEGScore for values less than 0 by max_negative_eeg (extreme negative)
mesh_points['NormalizedEEG'] = mesh_points['EEGScore'].apply(
    lambda x: x / abs(max_negative_eeg) if x < 0 else x / max_positive_eeg
)

# Calculate CombinedScore as the product of SaliencyScore and NormalizedEEG
mesh_points['CombinedScore'] = mesh_points['SaliencyScore'] * mesh_points['NormalizedEEG']

# Separate negative and positive values for CombinedScore
negative_values_combined = mesh_points['CombinedScore'][mesh_points['CombinedScore'] < 0]
positive_values_combined = mesh_points['CombinedScore'][mesh_points['CombinedScore'] > 0]

# Get the extreme negative and positive values for CombinedScore
max_combined_negative = negative_values_combined.min()  # Most negative combined score
max_combined_positive = positive_values_combined.max()  # Most positive combined score

# Normalize CombinedScore similarly
mesh_points['NormalizedCombinedScore'] = mesh_points['CombinedScore'].apply(
    lambda x: x / abs(max_combined_negative) if x < 0 else x / max_combined_positive
)
############################################################################################################################################
print(mesh_points[['x', 'y', 'z', 'SaliencyScore','NormalizedScore', 'EEGScore', 'NormalizedEEG', 'CombinedScore', 'NormalizedCombinedScore']].head(30))
mesh_points[['x', 'y', 'z', 'SaliencyScore','NormalizedScore', 'Labels','EEGScore','NormalizedEEG', 'NormalizedCombinedScore']].to_csv('rect15__combined_mulitply_score.csv', index=False)

print("CSV file 'curved5_score_with_eeg.csv' has been saved.")
############################################################################################################################################

----------------------------------------------------------------------END OF PROCESSING --------------------------------------------------------------------------------